In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_original = pd.read_csv('raw_data/train.csv')
data_original.head(5)

,Loan.ID,Current.Loan.Amount,Term,Credit.Score,Years.in.current.job,Home.Ownership,Annual.Income,Purpose,Monthly.Debt,Years.of.Credit.History,Months.since.last.delinquent,Number.of.Open.Accounts,Number.of.Credit.Problems,Current.Credit.Balance,Maximum.Open.Credit,Bankruptcies,Tax.Liens,Loan.Status
0,0,17879,Short Term,739.0,6 years,Home Mortgage,95357.0,Debt Consolidation,1509.82,34.4,5.0,26,0,23986,40313,0.0,0.0,1
1,1,99999999,Long Term,619.0,6 years,Rent,54406.0,Debt Consolidation,1065.45,12.7,NaN,8,0,22938,31250,0.0,0.0,1
2,3,99999999,Short Term,738.0,less than 1 year,Rent,40480.0,Medical Bills,573.47,32.7,60.0,9,1,3963,5669,1.0,0.0,1
3,4,11200,Short Term,738.0,4 years,Rent,53965.0,Debt Consolidation,1358.11,15.8,NaN,8,1,11782,25951,1.0,0.0,1
4,5,3608,Short Term,731.0,10+ years,Home Mortgage,47709.0,Home Improvements,339.93,16.1,47.0,6,0,22909,25568,0.0,0.0,1


## 1. CLEANING

In [3]:
# (1) Deleting rows with Current.Loan.Amount equal to 99999999
data = data_original[data_original['Current.Loan.Amount'] != 99999999].copy()

In [4]:
# (2) Deleting rows where Credit.Score and Annual.Income are NA
data = data[data['Credit.Score'].notna()]
data = data[data['Annual.Income'].notna()]

In [5]:
# (3) Na coluna Home.Ownership, HaveMortgage substituido por Home Mortgage
data['Home.Ownership'] = data['Home.Ownership'].replace(['HaveMortgage'], 'Home Mortgage')  

In [6]:
# (4) If Years.in.current.job is null, fill out with "less than 1 year"
data['Years.in.current.job'] = data['Years.in.current.job'].fillna('less than 1 year')
data['Years.in.current.job'] = data['Years.in.current.job'].replace(['less than  1 year'], 'less than 1 year')  

In [7]:
# (5) If Bankruptcies are null, substitute by 0
data['Bankruptcies'] = data['Bankruptcies'].fillna(0)

In [8]:
# (6) If Tax.Liens are null, substitute by 0
data['Tax.Liens'] = data['Tax.Liens'].fillna(0)

In [9]:
# (7) If Credit.Score > 850 (maximum), divive by 10 (assuming there was a typing error)
#(e.g., instead of typing 750, person typed 7500)

def div(x):
    if x > 850:
        return x / 10
    else:
        return x

data['Credit.Score'] = data['Credit.Score'].apply(div)

## 2. ENCODING

In [10]:
#(1) Number of open accounts: buckets (0-10, 11-20, mais de 20)
data['Number.of.Open.Accounts.Buckets'] = data['Number.of.Open.Accounts']

def encoding_noab(x):
    if x < 10:
        return '0-10'
    elif x > 20:
        return 'mais de 20'
    else:
        return '11-20'

data['Number.of.Open.Accounts.Buckets'] = data['Number.of.Open.Accounts.Buckets'].apply(encoding_noab)
data.drop(columns='Number.of.Open.Accounts')

,Loan.ID,Current.Loan.Amount,Term,Credit.Score,Years.in.current.job,Home.Ownership,Annual.Income,Purpose,Monthly.Debt,Years.of.Credit.History,Months.since.last.delinquent,Number.of.Credit.Problems,Current.Credit.Balance,Maximum.Open.Credit,Bankruptcies,Tax.Liens,Loan.Status,Number.of.Open.Accounts.Buckets
0,0,17879,Short Term,739.0,6 years,Home Mortgage,95357.0,Debt Consolidation,1509.82,34.4,5.0,0,23986,40313,0.0,0.0,1,mais de 20
3,4,11200,Short Term,738.0,4 years,Rent,53965.0,Debt Consolidation,1358.11,15.8,NaN,1,11782,25951,1.0,0.0,1,0-10
4,5,3608,Short Term,731.0,10+ years,Home Mortgage,47709.0,Home Improvements,339.93,16.1,47.0,0,22909,25568,0.0,0.0,1,0-10
5,7,7877,Short Term,738.0,5 years,Home Mortgage,47707.0,Home Improvements,504.90,14.5,70.0,1,1837,6269,1.0,0.0,1,11-20
6,8,4732,Long Term,701.0,10+ years,Rent,62721.0,other,296.87,20.2,51.0,0,0,0,0.0,0.0,0,0-10
7,10,3469,Short Term,735.0,5 years,Rent,25135.0,Debt Consolidation,354.83,9.1,NaN,0,3646,6840,0.0,0.0,0,0-10
8,11,10031,Short Term,712.0,6 years,Rent,150465.0,other,1199.96,12.5,20.0,0,17885,22583,0.0,0.0,1,0-10
10,15,20044,Short Term,741.0,3 years,Home Mortgage,91200.0,Debt Consolidation,703.76,14.9,NaN,0,20523,25369,0.0,0.0,1,0-10
13,18,4836,Short Term,723.0,10+ years,Home Mortgage,75563.0,Debt Consolidation,585.61,13.4,NaN,0,22361,30341,0.0,0.0,1,11-20
14,19,6162,Short Term,684.0,10+ years,Home Mortgage,74154.0,Debt Consolidation,1044.33,15.7,56.0,0,4967,14395,0.0,0.0,0,11-20


In [11]:
#(2) Calculating current credit balance minus current loan amount
#if 'Credit.Minus.Loan'> 0, then, 1 (esta pedindo emprestimo menor do que tem de saldo disponivel, entao, ok); otherwise, 0

data['Credit.Minus.Loan'] = data['Current.Credit.Balance'] - data['Current.Loan.Amount']

def encoding_cml(x):
    if x > 0:
        return 1
    else:
        return 0

data['Credit.Minus.Loan'] = data['Credit.Minus.Loan'].apply(encoding_cml)

In [12]:
#(3) Number of Number.of.Credit.Problems: 0, 1 ou mais de 1
data['Number.of.Credit.Problems.Buckets'] = data['Number.of.Credit.Problems']

def encoding_nocp(x):
    if x ==0:
        return 0
    elif x ==1:
        return 1
    else:
        return 'mais de 1'

data['Number.of.Credit.Problems.Buckets'] = data['Number.of.Credit.Problems.Buckets'].apply(encoding_nocp)

In [13]:
# (4) Years.in.current.job - STRINGS CONVERTIDAS P/ INTEIROS DE 0 A 10 SENDO 'less than 1 year' E '10+ years' RESPECTIVAMENTE NO CAMPO NOVO 'yicj_enc', 

data['Years.in.current.job.enc'] = data['Years.in.current.job'].map({'less than 1 year': 0,
                                                             '1 year': 1,
                                                             '2 years': 2,
                                                             '3 years': 3,
                                                             '4 years': 4,
                                                             '5 years': 5,
                                                             '6 years': 6,
                                                             '7 years': 7,
                                                             '8 years': 8,
                                                             '9 years': 9,
                                                             '10+ years': 10})


In [14]:
# (5) Tipo de credito: a) Debt consolidation b) Home improvement c) Business loan d) Buy a car e) Medical bills f) Buy a house g) Others (juntar other + Other + o que sobrou, como moving, etc)

pur_cat_oth = ['other', 'moving', 'small_business', 'Take a Trip', 'major_purchase',
               'wedding', 'Educational Expenses', 'vacation', 'renewable_energy']

data['Purpose.Categories'] = data['Purpose']

for cat_oth in pur_cat_oth:
    data.loc[data['Purpose.Categories'] == cat_oth, 'Purpose.Categories'] = 'Other'

In [15]:
# (6) Tax.Liens 0 or 1:  TL==0 = 0 E TL>=1 = 1 NO CAMPO NOVO 'Tax.Liens.enc'

def encode_tl(x):
    if x == 0:
        x_enc = 0
    else:
        x_enc = 1
    return x_enc

data['Tax.Liens.enc'] = data['Tax.Liens'].map(encode_tl)

In [16]:
# (7) Criacao feature 'Leverage'

data['Leverage'] = (data['Monthly.Debt'] * 12) / data['Annual.Income']

In [17]:
# (8) Buckets para Credit.Score:
#Poor (579 and below)
#Below Average (580-619)
#Average (620-659)
#Good (660-719)
#Excellent (720 and above)

def encode_cs(x):
    if x >= 720:
        x_enc = 'Excellent'
    elif x >= 660:
        x_enc = 'Good'
    elif x >= 620:
        x_enc = 'Average'
    elif x >= 580:
        x_enc = 'Below Average'
    else:
        x_enc = 'Poor'
    return x_enc

data['Credit.Score.Buckets'] = data['Credit.Score'].map(encode_cs)

In [18]:
# (9) Years.of.Credit.History: criar 4 buckets (0-10; 10.01-20; 20.01-30; > 30.01)

data['Years.of.Credit.History.enc'] = pd.cut(x=data['Years.of.Credit.History'],
                              bins=[-1, 10, 20, 30, 100],
                              labels=['0 a 10', '10.1 a 20', '20.1 a 30', '30+'])


In [19]:
# (10) Bankruptcies: ja decretou falencia? Sim (> = 1) ou nao (0)

def encode_ban(x):
    if x == 0:
        x_enc = 0
    else:
        x_enc = 1
    return x_enc

data['Bankruptcies.enc'] = data['Bankruptcies'].map(encode_ban)

In [20]:
# (11) Months.since.last.delinquent: 
#Any delinquency in the past 3 yrs? Dividir meses por 12 --> se o resultado for menor que 3 (0 a 3), resultado eh sim e vira "1". Se for > 3 ou NA, vira "0".

data['Years.since.last.delinquent'] = data['Months.since.last.delinquent'] / 12
data.loc[data['Years.since.last.delinquent'].isnull(), 'Years.since.last.delinquent'] = 100

def encode_msld(x):
    if x <= 3:
        return 1
    else:
        return 0

data['Years.since.last.delinquent'] = data['Years.since.last.delinquent'].map(encode_msld)

In [21]:
# (12) Current loan amount / Annual income
data['Loan.vs.Income'] = data['Current.Loan.Amount'] / data['Annual.Income']

In [22]:
data.dtypes

Loan.ID                                 int64
Current.Loan.Amount                     int64
Term                                   object
Credit.Score                          float64
Years.in.current.job                   object
Home.Ownership                         object
Annual.Income                         float64
Purpose                                object
Monthly.Debt                          float64
Years.of.Credit.History               float64
Months.since.last.delinquent          float64
Number.of.Open.Accounts                 int64
Number.of.Credit.Problems               int64
Current.Credit.Balance                  int64
Maximum.Open.Credit                     int64
Bankruptcies                          float64
Tax.Liens                             float64
Loan.Status                             int64
Number.of.Open.Accounts.Buckets        object
Credit.Minus.Loan                       int64
Number.of.Credit.Problems.Buckets      object
Years.in.current.job.enc          

## Dataframe to be used in visualization (open another notebook) & transformers

In [23]:
#From data, dropped some columns to get the df to be used in next phase of workd
df = data.drop(columns=['Loan.ID', 'Credit.Score', 'Years.in.current.job',
                       'Purpose', 'Monthly.Debt', 'Years.of.Credit.History',
                       'Months.since.last.delinquent', 'Number.of.Open.Accounts',
                       'Number.of.Credit.Problems', 'Maximum.Open.Credit',
                       'Bankruptcies', 'Tax.Liens'])
df.head(5)

,Current.Loan.Amount,Term,Home.Ownership,Annual.Income,Current.Credit.Balance,Loan.Status,Number.of.Open.Accounts.Buckets,Credit.Minus.Loan,Number.of.Credit.Problems.Buckets,Years.in.current.job.enc,Purpose.Categories,Tax.Liens.enc,Leverage,Credit.Score.Buckets,Years.of.Credit.History.enc,Bankruptcies.enc,Years.since.last.delinquent,Loan.vs.Income
0,17879,Short Term,Home Mortgage,95357.0,23986,1,mais de 20,1,0,6,Debt Consolidation,0,0.190000,Excellent,30+,0,1,0.187495
3,11200,Short Term,Rent,53965.0,11782,1,0-10,1,1,4,Debt Consolidation,0,0.301998,Excellent,10.1 a 20,1,0,0.207542
4,3608,Short Term,Home Mortgage,47709.0,22909,1,0-10,1,0,10,Home Improvements,0,0.085501,Excellent,10.1 a 20,0,0,0.075625
5,7877,Short Term,Home Mortgage,47707.0,1837,1,11-20,0,1,5,Home Improvements,0,0.127000,Excellent,10.1 a 20,1,0,0.165112
6,4732,Long Term,Rent,62721.0,0,0,0-10,0,0,10,Other,0,0.056798,Good,20.1 a 30,0,0,0.075445


In [24]:
df.isnull().sum()

Current.Loan.Amount                  0
Term                                 0
Home.Ownership                       0
Annual.Income                        0
Current.Credit.Balance               0
Loan.Status                          0
Number.of.Open.Accounts.Buckets      0
Credit.Minus.Loan                    0
Number.of.Credit.Problems.Buckets    0
Years.in.current.job.enc             0
Purpose.Categories                   0
Tax.Liens.enc                        0
Leverage                             0
Credit.Score.Buckets                 0
Years.of.Credit.History.enc          0
Bankruptcies.enc                     0
Years.since.last.delinquent          0
Loan.vs.Income                       0
dtype: int64

In [26]:
df.to_csv(r"C:\Users\m_day\code\mdayoub493\wanna_buy_house\notebooks\raw_data\data_cleaned_encoded.csv", index = False, header=True)